# GLOFAS historical streamflows: visualization

In this tutorial, we will visualise historical streamflow data from the GLObal Flood Awareness System that is operated by the European Commission.

## Preliminaries
This tutorial requires the presence of various Python libraries that may need to be installed.

In [3]:
!pip install xarray
!pip install cfgrib
!pip install matplotlib
!pip install numpy


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Once installed, the various packages need to be loaded into memory.

In [6]:
import os
import cfgrib

# Libraries for working with multidimensional arrays
import numpy as np
import xarray as xr

# Libraries for plotting and visualising data
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

RuntimeError: Cannot find the ecCodes library

## Location of the data
This tutorial assumes that data has already been downloaded. If used from within github.com, online data can be used.

Using `xarray`, the dataset is accessed: 

In [5]:
glofas_data = xr.open_dataset(f'./glofas_historical_simulation_june_2012_2022.grib')
glofas_data

C:\Users\verkade\AppData\Local\Programs\Python\Python310\lib\site-packages\xarray\backends\plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
C:\Users\verkade\AppData\Local\Programs\Python\Python310\lib\site-packages\xarray\backends\cfgrib_.py:54: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


RuntimeError: Cannot find the ecCodes library

## Plotting function
Below function is used to plot our data. Note that this function is taken from a script that was written by ECMWF (https://ecmwf-projects.github.io/copernicus-training-c3s/glofas-bangladesh-floods.html#explore-and-view-the-data).

In below function, the extent has the convention `[x0,x1,y0,y1]`. This convention differs from the one needed to download data from the CDS!

In [ ]:
# Create a simple plotting function that we can use throughout this notebook
def plot_map(
    plot_data,
    title='',
    cbar_label='',
    cmap='PuBu',
    extent=[-180, 180, -90, 90], #[x0,x1,y0,y1] = [West,East,South,North]
    **pcolorkwargs
):
    # Populate the title and cbar_label with attributes from the plot_data if they have not been
    #  explicitly specified
    title = title or plot_data.attrs.get('long_name', title)
    cbar_label = cbar_label or plot_data.attrs.get('units', cbar_label)

    # Create a figure with a cartopy projection assigned which allows plotting geospatial data
    fig, ax = plt.subplots(
        1, 1, figsize = (18, 9), subplot_kw={'projection': ccrs.PlateCarree()}
    )

    # Plot the data on our figure
    im = ax.pcolormesh(
        plot_data.longitude, plot_data.latitude, plot_data, cmap=cmap, **pcolorkwargs
    )

    # Add some additional features
    ax.set_title(title, fontsize=16)
    ax.gridlines(draw_labels=False, linewidth=1, color='gray', alpha=0.5, linestyle='--') 
    ax.coastlines(color='black')

    # Add country borders in red
    ax.add_feature(cfeature.BORDERS, edgecolor='black', lw=1.5, ls=":")

    # Set the plot domain/extent
    ax.set_extent(extent, crs=ccrs.PlateCarree())

    # Add a colour bar
    cbar = plt.colorbar(im,fraction=0.04, pad=0.01)
    cbar.set_label(cbar_label, fontsize=12) 

## Plot mean streamflow
First, the average streamflow is plotted. This is an average over all data that is present in the file we accessed.

In [ ]:
mean_data = glofas_data.dis24.mean(dim='time')
mean_data = mean_data.assign_attrs(**glofas_data.dis24.attrs)
plot_map(
    mean_data,
    vmax=1e3, vmin=0,
    cbar_label = "m³ s⁻¹",
    extent=[-2, 11, -9, 4], #[x0,x1,y0,y1] = [West,East,South,North]
    title = 'Average streamflow'
)

## Plot data for a single location
The data file contains gridded data. From the grid, we extract data for a single location. We then plot the full timeseries.

In [ ]:
glofas_data.dis24.sel(latitude=0, longitude=10, method='nearest').plot()

## Compute statistics
On the assumption that we have a long timeseries available, we can compute various return intervals.

@Include this, based on c:\Users\verkade\OneDrive - Stichting Deltares\_gloffis\20211230-derive_stats_example.ipynb

Plot an ensemble forecast

In [ ]:
Q = sim_grid.Q.sel(x=5.694536, y=50.815519, method='nearest')
fig, ax = plt.subplots()
for i in range(Q.shape[0]):
  ax.plot(Q['time'].values, Q.values[:,i], color='gray', alpha=0.5)
ax.set_xlabel('Time')
ax.set_ylabel('Streamflow [m3/s]')
ax.set_title('Plume plot')
plt.show()